In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from unidecode import unidecode

url_setores = 'https://investidor10.com.br/setores/'

def buscar_ativos(url, setor, subsetor):
    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "select2-sector-container")))
    time.sleep(5)

    try:
        titulos = driver.find_elements(By.CLASS_NAME, "actions-title")
        titulos_text = [titulo.text for titulo in titulos]
    except:
        titulos_text = False

    try:
        pag = driver.find_elements(By.CLASS_NAME, "pagination-link")
        pag_num = [int(p.text) for p in pag if p.text.isnumeric()]
    except:
        pag_num = False

    if pag_num:
        for i in range(2, pag_num[-1]+1):
            driver.get(f'{url}?page={i}')
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "actions-title")))
            time.sleep(5)

            titulos = driver.find_elements(By.CLASS_NAME, "actions-title")
            titulos_text += [titulo.text for titulo in titulos]

    driver.quit()

    if titulos_text:
        tickers = pd.DataFrame(titulos_text, columns=['ticker'])
        tickers['ticker'] = tickers['ticker'].str.extract(r"(\w*)$")[0]
        tickers['setor'] = setor
        tickers['subsetor'] = subsetor
    else:
        tickers = pd.DataFrame()

    return tickers

def buscar_setores(url):
    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "actions-title")))
    time.sleep(2)

    select = driver.find_element(By.ID, "select2-sector-container")
    select.click()
    time.sleep(1)

    setores = driver.find_elements(By.CLASS_NAME, "select2-results__option")
    setores_text = [setor.text for setor in setores]

    driver.quit()

    return setores_text

def buscar_subsetores(url, setor):
    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "actions-title")))
    time.sleep(2)

    setor_select = driver.find_element(By.ID, "select2-sector-container")

    action = ActionChains(driver)
    action.move_to_element(setor_select).perform()
    time.sleep(1)

    select = driver.find_element(By.ID, "select2-subsector-container")
    select.click()
    time.sleep(1)

    subsetores = driver.find_elements(By.CLASS_NAME, "select2-results__option")
    subsetores_text = [(setor, subsetor.text) for subsetor in subsetores]

    driver.quit()

    df = pd.DataFrame(subsetores_text, columns=['setor', 'subsetor'])

    return df

def buscar_ativos_por_subsetor(url, setor, subsetor):
    setor = unidecode(setor).replace(' ', '-').replace(',', '').lower()
    subsetor = unidecode(subsetor).replace(' ', '-').replace(',', '').lower()

    url_subsetor = f'{url}{setor}/{subsetor}'

    tickers = buscar_ativos(url_subsetor, setor, subsetor)

    return tickers

In [13]:
setores = pd.read_csv('./data/ativos/setor/setores_acentuados.csv')['setor'].to_list()

subsetores = pd.DataFrame(columns=['setor', 'subsetor'])
for setor in setores:
    print(setor.title())
    setor = unidecode(setor).replace(' ', '-').lower()
    url_subsetores = url_setores + setor
    subsetores_group = buscar_subsetores(url_subsetores, setor)
    subsetores = pd.concat([subsetores, subsetores_group])

subsetores = subsetores[subsetores['subsetor'] != 'TODOS']
subsetores.to_csv('./data/ativos/setor/subsetores2.csv', index=False)

Bens Industriais


KeyboardInterrupt: 

In [15]:
from tqdm import tqdm

subsetores = pd.read_csv('./data/ativos/setor/subsetores.csv')

tickers = pd.DataFrame(columns=['ticker', 'setor', 'subsetor'])
for row in tqdm(subsetores.itertuples(index=False), total=len(subsetores)):
    print(row.setor.title(), row.subsetor.title())
    tickers_group = buscar_ativos_por_subsetor(url_setores, row.setor, row.subsetor)
    if tickers_group.empty:
        print('Nenhum ativo encontrado')
        continue
    tickers = pd.concat([tickers, tickers_group])

  0%|          | 0/45 [00:00<?, ?it/s]

Bens Industriais Comércio


  2%|▏         | 1/45 [00:16<12:09, 16.59s/it]

Bens Industriais Construção E Engenharia


  4%|▍         | 2/45 [00:33<11:53, 16.59s/it]

...

 96%|█████████▌| 43/45 [11:59<00:34, 17.06s/it]

Utilidade Publica Energia Elétrica


 98%|█████████▊| 44/45 [12:16<00:16, 16.99s/it]

Utilidade Publica Gás


100%|██████████| 45/45 [12:32<00:00, 16.73s/it]


In [18]:
tickers['setor'] = tickers['setor'].str.replace('-', ' ').str.title()
tickers['subsetor'] = tickers['subsetor'].str.replace('-', ' ').str.title()
tickers.to_csv('./data/ativos/tickers2.csv', index=False)